In [1]:
import numpy
import numpy as np
import seaborn as sns
import scipy.stats as stats
import matplotlib.pyplot as plt
import pandas as pd
from pandas import *
from numpy.linalg import inv
import pylab
import statistics
import time
import math
import heapq
from itertools import product

In [10]:
#Make modifications here
#------------------------------------------------------------------------------------------
nC=6; nN=6 #Choose
RandomLambda = 0 #Choose
#------------------------------------------------------------------------------------------

if(RandomLambda == 0):#If Lambda is been supplied from a file
    FileName = 'UIS_FromMarriageAlgo_PythonCode_L6.txt'   #--------------------------Change
    Lambda = np.loadtxt('Lambda_1_6_6.txt')
    LambdaC = Lambda[0,:]
    LambdaN = Lambda[1,:]
    print('LambdaC \n', LambdaC, '\nLambdaN \n', LambdaN)
else:
    Seed = 13      # ----- Choose vale
    FileName = 'UIS_FromMarriageAlgo_PythonCode_L6_Seed13.txt'   #-------------------Change
    np.random.seed(Seed)
    LambdaC = np.random.uniform(10,100,nC*nN)
    LambdaN = np.random.uniform(10,100,nC*nN)
    print('LambdaC \n', LambdaC, '\nLambdaN \n', LambdaN)

LambdaC 
 [47.439942 78.076943 93.739284 68.852708 75.039438 44.507408 89.623644
 68.636672 33.620769 77.802027 16.541729 90.789064 56.466278 32.2335
 86.155022 13.11149  71.136679 15.45079  52.995859 94.122363 38.712964
 10.692837 33.953006 34.937361 24.985648 49.277027 76.260208 18.165961
 54.457901 51.800125 47.087269 91.928874 57.677263 63.007896 92.243572
 89.971499] 
LambdaN 
 [18.27684  57.67302  44.515186 16.015801 70.403444 66.847124 56.747475
 31.399699 78.597823 91.828731 34.543897 34.741616 42.333847 53.786563
 84.786881 99.193363 78.984528 44.572794 95.298785 91.418777 73.107153
 42.880481 98.797791 66.658908 76.247371 98.429524 30.987539 55.407063
 14.507559 57.42359  37.568965 79.318409 58.357359 71.809543 26.049494
 84.503559]


In [11]:
#Make the lambda matrix
#LambdaMat[i,j,0] = Lambda_ij_C
#LambdaMat[i,j,1] = Lambda_ij_N
LambdaMat = numpy.zeros(shape=(nC,nN,2))
counter = 0
for i in range(nC):
    for j in range(nN):
        LambdaMat[i,j,0] = LambdaC[counter]
        LambdaMat[i,j,1] = LambdaN[counter]
        counter += 1
LambdaMat

array([[[47.439942, 18.27684 ],
        [78.076943, 57.67302 ],
        [93.739284, 44.515186],
        [68.852708, 16.015801],
        [75.039438, 70.403444],
        [44.507408, 66.847124]],

       [[89.623644, 56.747475],
        [68.636672, 31.399699],
        [33.620769, 78.597823],
        [77.802027, 91.828731],
        [16.541729, 34.543897],
        [90.789064, 34.741616]],

       [[56.466278, 42.333847],
        [32.2335  , 53.786563],
        [86.155022, 84.786881],
        [13.11149 , 99.193363],
        [71.136679, 78.984528],
        [15.45079 , 44.572794]],

       [[52.995859, 95.298785],
        [94.122363, 91.418777],
        [38.712964, 73.107153],
        [10.692837, 42.880481],
        [33.953006, 98.797791],
        [34.937361, 66.658908]],

       [[24.985648, 76.247371],
        [49.277027, 98.429524],
        [76.260208, 30.987539],
        [18.165961, 55.407063],
        [54.457901, 14.507559],
        [51.800125, 57.42359 ]],

       [[47.087269, 37.568965]

In [12]:
#Obtain the Rank List of Species on each Nutrient
#RankC[i] lists the N index of species according to their rank on C_i (high to low)
#RankN[j] lists the C index of species according to their rank on N_j (high to low)
#**remember that index starts from 0. So rank 0 is heighest.
#RankC = np.zeros(shape=(nC,nN))
RankC = np.empty([nC,nN+1], dtype=int)
RankC[:,nN] = nN
for i in range(nC):
    for j in range(nN):
        Temp = LambdaMat[i,:,0]
        RankC[i,0:nN] = heapq.nlargest(nN, range(len(Temp)), key=Temp.__getitem__)

RankN = np.empty([nN,nC+1], dtype=int)
RankN[:,nC] = nC
#RankN = np.zeros(shape=(nC,nN))
for j in range(nN):
    for i in range(nC):
        Temp = LambdaMat[:,j,1]
        RankN[j,0:nC] = heapq.nlargest(nC, range(len(Temp)), key=Temp.__getitem__)

In [13]:
print("RankC\n", RankC, "\nRankN\n", RankN)

RankC
 [[2 1 4 3 0 5 6]
 [5 0 3 1 2 4 6]
 [2 4 0 1 5 3 6]
 [1 0 2 5 4 3 6]
 [2 4 5 1 0 3 6]
 [4 1 5 3 2 0 6]] 
RankN
 [[3 4 1 2 5 0 6]
 [4 3 5 0 2 1 6]
 [2 1 3 5 0 4 6]
 [2 1 5 4 3 0 6]
 [3 2 0 1 5 4 6]
 [5 0 3 4 2 1 6]]


In [14]:
#Choose the rank of the species allocated on C
#and make a copy of the Rank Matrix
ChoseRankC = np.empty([nC], dtype=int)#Choose the rank of the species allocated on C

#This will give a sequence of numbers from 0 - nN
JJ = np.arange(nN+1)

#This will create a 2D matrix with ALL possible combinations of the numbers in JJ of size nC
#Each time we will choose one row from this 2D matrix as the rank choice allocated on Carbon
WWW = list(product(JJ,repeat = nC))

NoOfUIS = 0
States = []

for Times in range(pow(nC+1,nN)):
    for p in range(nC):
        ChoseRankC[p] = WWW[Times][p] #Chose the rank choice on row number=Times
    
    #print(ChoseRankC)
    TempRankC = np.copy(RankC)
    TempRankN = np.copy(RankN)
    Matrix = np.zeros([nC, nN], dtype = int) #This will be the matrix of states
    
    #554 means this one has been chosen on the C
    for i in range(nC):
        TempRankC[i,ChoseRankC[i]] = 554 #554 means this one has been chosen on the C
        if(RankC[i,ChoseRankC[i]] < nN):
            index = np.where(TempRankN[RankC[i,ChoseRankC[i]],:] == i)
            TempRankN[RankC[i,ChoseRankC[i]],index] = 554
            
     #-1000 means this one can't use C (both as limiting or non-limiting)
    for i in range(nC):
        for k in range(ChoseRankC[i]+1,nC+1,1):
            TempRankC[i,k] = -1000 #-1000 means this one can't use C (both as limiting or non-limiting)
            if(RankC[i,k] < nN):
                index = np.where(TempRankN[RankC[i,k],:] == i)
                TempRankN[RankC[i,k],index] = -1000
                
    #If there is a 554 in any row of TempRankN then in that row check that:
    #If there is any valid C index above the lowest ranked 554
    #i.e., search for valid C index before the last 554 in TempRankN
    UIS = 1
    for i in range(nN):
        if(UIS == 1):
            #print("for", i)
            if(554 in TempRankN[i]):
                item_index = np.where(TempRankN[i]==554)
                #print(i, item_index)
                Smallest554 = max(item_index[0])
                #print("Smallest554", Smallest554)
                for j in range(nC):
                    pos = np.where(TempRankN[i]==j)
                    if(np.size(pos) > 0):
                        aaa = max(pos[0])
                        #print(pos, j, aaa)
                        if(pos[0] < Smallest554):
                            #print("No UIS")
                            UIS = 0
    if(UIS == 1):
        #print("There is a UIS here")
        
        #Searching for Species limited by C
        for i in range(nC):
            pos=np.where(TempRankC[i,0:nN+1] == 554)
            pos = max(pos)
            #print(i,pos)
            if(pos < nN):
                #print(i,pos,RankC[i,pos])
                Matrix[i,RankC[i,pos]] = -1
        #print("Matrix\n",Matrix)
    
        #Searching for Species limited by N
        for i in range(nN):
            for j in range(nC):
                if((TempRankN[i,j] != 554) and (TempRankN[i,j] != -1000)):
                    #print(i,j,TempRankN[i,j])
                    Matrix[TempRankN[i,j],i] = 1
                    break
                
        G = np.zeros([nC*nN], dtype=int)
        counter = 0
        for i in range(nC):
            for j in range(nN):
                G[counter] = Matrix[i,j]
                counter += 1
        #print("UIS", NoOfUIS+1, *G, sep=' ')
        States.append(G)
        NoOfUIS += 1

In [15]:
np.savetxt(FileName, States, delimiter='\t',  fmt = '%d')

In [16]:
NoOfUIS

1211

In [17]:
np.shape(States)

(1211, 36)